#### A notebook which writes @Haoyu Ji's HF attributes to the MHPI-Spatial S3 bucket

In [1]:
from pathlib import Path

import geopandas as gpd
import icechunk as ic
import numpy as np
import pandas as pd
import xarray as xr
import zarr
from icechunk.xarray import to_icechunk


In [2]:
import boto3
import os

profile_name = "CIROH_USER"  # Replace with your AWS credentials file profile name
os.environ['AWS_PROFILE'] = profile_name

In [3]:
file_path = Path("/projects/mhpi/data/hydrofabric/v2.2/attributes.nc")
bucket="mhpi-spatial"
prefix="hydrofabric_v2.2_attributes"
commit="initial attr commit"

In [4]:
_ds = xr.open_mfdataset(file_path, engine='netcdf4')
_ds

<xarray.Dataset> Size: 225MB
Dimensions:            (divide_id: 839543)
Coordinates:
  * divide_id          (divide_id) <U11 37MB 'cat-1068193' ... 'cat-3014411'
Data variables: (12/28)
    FW                 (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>
    HWSD_clay          (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>
    HWSD_sand          (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>
    T_clay             (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>
    uparea             (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>
    T_gravel           (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>
    ...                 ...
    ETPOT_Hargr        (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>
    meanTa             (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>
    SoilGrids1km_clay  (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>
    snow_fraction      (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>
    aridity            (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>
    NDVI               (divide_id) float64 7MB dask.array<chunksize=(839543,), meta=np.ndarray>

In [5]:
ds = _ds.drop_duplicates(dim="divide_id")

In [6]:
ds

<xarray.Dataset> Size: 222MB
Dimensions:            (divide_id: 828210)
Coordinates:
  * divide_id          (divide_id) <U11 36MB 'cat-1068193' ... 'cat-3014411'
Data variables: (12/28)
    FW                 (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>
    HWSD_clay          (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>
    HWSD_sand          (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>
    T_clay             (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>
    uparea             (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>
    T_gravel           (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>
    ...                 ...
    ETPOT_Hargr        (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>
    meanTa             (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>
    SoilGrids1km_clay  (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>
    snow_fraction      (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>
    aridity            (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>
    NDVI               (divide_id) float64 7MB dask.array<chunksize=(828210,), meta=np.ndarray>

In [7]:
storage_config = ic.s3_storage(
bucket=bucket, prefix=prefix, region="us-east-2", from_env=True
)
repo = ic.Repository.open_or_create(storage_config)
session = repo.writable_session("main")
to_icechunk(ds, session)
msg = f"Uploaded attrs to S3"
latest_snapshot = session.commit(msg)

/home/tbindas/tbindas/.conda/envs/nwm_batcher/lib/python3.12/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/home/tbindas/tbindas/.conda/envs/nwm_batcher/lib/python3.12/site-packages/zarr/core/array.py:4257: UserWarning: The dtype `<U11` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  meta = AsyncArray._create_metadata_v3(
